# P2 submission
1. Dataset analyzed: Titanic<br>
2. Question posed: Did the variables below influenced the chance of an individual to survive the Titanic accident?<br>
Pclass<br>
Age<br>
Fare<br>
Sex
3. What I did to investigate those questions? it is detailed step by step below

## Variables
The dependent variable is **Survived** (whether or not a given passanger survived the accident). I'll statistically test if each of the independent variables listed in the item 2 above.

## Hypotheses:
**Ho: μsample = μsurvived → μsample - μsurvived = 0
Ha: μsample ≠ μsurvived → μsample - μsurvived ≠ 0**

## Statistical Test:
**Two-tailed test**
I've chosen the hypotheses above because I think that the objective of the test is to check if a given independent variable had influence over the dependent variable in any direction.

## Alpha Level
**α = 0.05** (0.025 in each tail) (confidence level = 95%)

In [1]:
#importing libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#loading dataset
filename = '/Users/leo/Dropbox/udacity_nanodegree/titanic_data.csv'
titanic_df = pd.read_csv(filename)

In [2]:
#checking the dataframe
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# adding a new SexMap numeric column to the dataframe to map the sex row string values to numeric. 
# female = 1, male = 0.
sex_map = {'male': 0, 'female': 1}

sex_map_column = titanic_df['Sex'].map(sex_map)      
titanic_df['SexMap'] = sex_map_column
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,SexMap
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [4]:
#checking correlation between Survived and other variables
titanic_df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,SexMap
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658,-0.042939
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,0.543351
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.131900
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,-0.093254
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,0.114631
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,0.245489
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,0.182333
SexMap,-0.042939,0.543351,-0.131900,-0.093254,0.114631,0.245489,0.182333,1.000000


In [5]:
# playing arround with the data to make up my mind (checking counts and proportion of survivals)
titanic_df_survivals = titanic_df[titanic_df['Survived'] == 1]
titanic_df_deaths = titanic_df[titanic_df['Survived'] == 0]

num_passengers = titanic_df['PassengerId'].count()
num_survivals = titanic_df_survivals['PassengerId'].size
num_deaths = titanic_df_deaths['PassengerId'].size

proportion_of_survivals = num_survivals / float(num_passengers)

print num_passengers
print num_survivals
print num_deaths
print num_survivals + num_deaths
print proportion_of_survivals


891
342
549
891
0.383838383838


In [7]:
# checking statistics about the whole sample 
# (not sure why I get a warning when I first run this cell - only happens the first time you execute it)
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,SexMap
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.352413
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200,0.000000
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


### One sample t-Test formula used below to calculate the t-statistic for each of the independent variables I want to check
$$t_s= \frac{\bar X - \mu _o}{\frac{S}{\sqrt{n}}}$$

In [8]:
# prints statistics about the whole sample grouped by Survived
titanic_df_groupby_survived = titanic_df.groupby(['Survived'])
def get_statistics(independent_variable_name):
#     print 'get_t_statistic ' + independent_variable_name
    n_independent_variable_name = titanic_df_groupby_survived.count().loc[1][independent_variable_name]
#     print n_independent_variable_name
    df_independent_variable_name = n_independent_variable_name - 1
#     print df_independent_variable_name

    mean_independent_variable_name = titanic_df_groupby_survived.mean().loc[1][independent_variable_name]
#     print mean_independent_variable_name

    std_independent_variable_name = titanic_df_groupby_survived.std(ddof=False).loc[1][independent_variable_name]
#     print std_independent_variable_name

    t_stat = (mean_independent_variable_name - titanic_df[independent_variable_name].mean())/(std_independent_variable_name/(float(n_independent_variable_name)**(0.5)))
#     print t_stat
    
    return (t_stat, n_independent_variable_name, df_independent_variable_name)

### Bellow I'l get statistics for each of the independent variables listed at the top: 
- Pclass
- Age
- Fare
- Sex

In [9]:
get_statistics('Pclass')

(-7.6874746908532243, 342, 341)

In [10]:
get_statistics('Age')

(-1.5465240571264192, 290, 289)

In [11]:
get_statistics('Fare')

(4.5027028839416285, 342, 341)

In [12]:
get_statistics('SexMap')

(13.051994145653103, 342, 341)

In [13]:
# I will use the same t-critical to draw the conclusions below because
# all independent variables have df values > 100 < 10000.
# Using t-critical for df=100 and p=0.025 (two-tail t-Test)
tcritical = (-1.984, 1.984)

# Conclusions
### Pclass:
-7.6874746908532243 < -1.984 ==> p < 0.025<br>
**Reject the Null Hypotheses**<br>
Passengers with lower class numbers had higher chances of surviving
![caption](pclass.gif)

### Age:
-1.5465240571264192 > -1.984 ==> p > 0.025<br>
**Accept the Null Hypotheses**<br>
Even though the number suggests a correlation between Age and chance of surviving it is not enough to meet ourconfidence criteria (α = 0.05, two-tailed t-Test)
![caption](age.gif)

### Fare:
4.5027028839416285 > 1.984 ==> p < 0.025<br>
**Reject the Null Hypotheses**<br>
Passengers that paid higher fares had higher chances of surviving
![caption](fare.gif)

### Sex:
13.051994145653103 > -1.984 ==> p < 0.025<br>
**Reject the Null Hypotheses**<br>
Females passengers had higher chances of surviving
![caption](sex.gif)

#### Resources
http://www.stat.ufl.edu/~athienit/Tables/Ttable.pdf<br>
http://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas<br>
http://stackoverflow.com/questions/117250/how-do-i-get-a-decimal-value-when-using-the-division-operator-in-python<br>
http://meta.math.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference<br>
http://www-users.cs.umn.edu/~ludford/stat_overview.htm<br>
https://en.wikibooks.org/wiki/Python_Programming/Basic_Math#Mathematical_Operators<br>
http://support.minitab.com/en-us/minitab/17/topic-library/basic-statistics-and-graphs/hypothesis-tests/tests-of-means/types-of-t-tests/<br>
http://stackoverflow.com/questions/13404468/t-test-in-pandas-python<br>
http://www.imathas.com/stattools/norm.html<br>
http://stackoverflow.com/questions/10628262/inserting-image-into-ipython-notebook-markdown